# Multilabel Classification

With multilabel classification we have two or more classifications for a single set of attributes. As a real life example, we could have a face detector that was trained to recognize  Jane, Alice and Bob. If it comes across than say (Alice, Bob) or (Alice, Bob, Dan) It would output a [0, 1, 1] in both case in recognition that Alice and Bob are present in the photo.


**Example**


In [1]:
import numpy as np
from os.path import join
from myutils.config import config
from mlxtend.data import loadlocal_mnist


data_root = join(config['data_dir'], 'digits v2')
X_train, y_train = loadlocal_mnist(images_path=join(data_root, 'train-images.idx3-ubyte'), 
                                   labels_path=join(data_root, 'train-labels.idx1-ubyte'))
X_test, y_test = loadlocal_mnist(images_path=join(data_root, 't10k-images.idx3-ubyte'), 
                                 labels_path=join(data_root, 't10k-labels.idx1-ubyte'))


shuffle_index_train = np.random.permutation(X_train.shape[0])
shuffle_index_test = np.random.permutation(X_test.shape[0])

X_train, y_train = X_train[shuffle_index_train], y_train[shuffle_index_train]
X_test, y_test = X_test[shuffle_index_test], y_test[shuffle_index_test]

As an example, ill create two different toy binary classifications. These two classification will be horizontally stacked to perform a multilabeled final class. We'll then train a k-nearest-neighbor classifier to learn a multilabeled classification.

In [2]:
from sklearn.neighbors import KNeighborsClassifier


y_train_7_or_more = (y_train >= 7)
y_train_odd = (y_train % 2 == 1)
y_train_multilabel = np.c_[y_train_7_or_more, y_train_odd]
knn_clf = KNeighborsClassifier()
knn_clf.fit(X_train, y_train_multilabel)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=5, p=2,
           weights='uniform')

In [3]:
# sample prediction: large but not odd
print(f'prediction: {knn_clf.predict([X_test[123]])}')
print(f'actual    : {y_test[123]}')

prediction: [[ True  True]]
actual    : 9


**How does this work?**

MY GUESS

If we can already understand how a single classification process works, then in the case of a multilabeled classifier can just generalize by individually training a classifier for each column of `y_train_multilabel`. Then process of `predict` would be to reference each classifier iteratively.

## Evaluation

One option is compute the average cross validated f1-score. Recall that the f1-score is a weighted aggregate of precision and recall on the classification. In our case, because this is a multilabeled classification, an f1-score would be computed per each `y_train_multilabel`, and the average would be computed.

Another option would be perform a weighted f1-score. For example, we could have more pictures of Alice than of Bob. This would mean that classifier would "learn" Alice better, and thus have a more accurate score of Alice than of Bob. In this way, it would make sense then to weight the classifier for Alice proportional to the frequency of Alice photos, and to the frequency of Bob photos. The score in this case wouldn't become skewed due to having barely trained on Bob photos.

In [4]:
from sklearn.model_selection import cross_val_predict


# y_train_knn_pred = cross_val_predict(knn_clf, X_train, y_train_multilabel, cv=3)
# f1_score(y_train, y_train_knn_pred, average="macro")

# Multioutput Classification

As a generalization of the above, lets us focus on the case where the output of a classifier is to reproduce a multi-class for a set of classes. As a real world example, suppose that we now wanted to detect a specific set of people in a photo. For each person, a classifier could identify person A, B, C... for every person in a photograph.

**Example**

Suppose we wanted to remove noise from an image. In this case, we have 28x28 pixeled image, where each pixel has a class of 255 color intensity/shades of black. The goal would be to taken noisey image X and output a clear image y which is a multilabeled classfication.


In [5]:
import numpy as np


# input X = noisey images, train and test sets
noise_matrix = np.random.randint(0, 100, (len(X_train), 784))
X_train_mod = X_train + noise_matrix
noise_matrix = np.random.randint(0, 100, (len(X_test), 784))
X_test_mod = X_test + noise_matrix

# output y clear images, train and test sets
y_train_mod = X_train
y_test_mod = X_test

In [6]:
# train a multilabled model
knn_clf.fit(X_train_mod, y_train_mod)
clean_digit = knn_clf.predict([X_test_mod[1]])

def plot_digit(digit_matrix):
    digit_matrix = digit_matrix.reshape(28, 28)
    plt.imshow(digit_matrix, cmap=matplotlib.cm.binary, interpolation="nearest")
    plt.axis("off")
    plt.show()

plot_digit(clean_digit)

NameError: name 'plt' is not defined